In [111]:
import pickle
import pandas as pd

In [112]:
df = pd.read_csv("data/heart.csv")
df.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


Abrimos todos los *transformers* que guardamos cuando hicimos el modelo

In [113]:
# encoding ChestPainType
with open('data/encodingChestPainType.pkl', 'rb') as chest:
    encoding_chestpain = pickle.load(chest)
    
# encoding Angina
with open('data/encodingExerciseAngina.pkl', 'rb') as angina:
    encoding_angina = pickle.load(angina)

    
# encoding Fasting
with open('data/encodingFastingBS.pkl', 'rb') as fasting:
    encoding_fasting = pickle.load(fasting)

# encoding Resting
with open('data/encodingRestingECG.pkl', 'rb') as resting:
    encoding_resting = pickle.load(resting)

# encoding Slope
with open('data/encodingST_Slope.pkl', 'rb') as slope:
    encoding_slope = pickle.load(slope)

# encoding Sex
with open('data/encodingSex.pkl', 'rb') as sex:
    encoding_sex = pickle.load(sex)

# estandarización
with open('data/estandarizacion.pkl', 'rb') as estandarizacion:
    estandarizacion = pickle.load(estandarizacion)


In [135]:
# cargamos el mejor modelo
with open('data/mejor_modelo.pkl', 'rb') as modelo:
    modelo = pickle.load(modelo)

Nos preparamos para hacer la predicción

In [115]:
# recordamos como eran los datos que nos pasaban de origen
df.sample(4)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
388,55,M,ASY,140,0,0,Normal,83,N,0.0,Flat,1
812,54,F,NAP,110,214,0,Normal,158,N,1.6,Flat,0
672,60,F,NAP,120,178,1,Normal,96,N,0.0,Up,0
96,43,M,ATA,142,207,0,Normal,138,N,0.0,Up,0


In [126]:
# le pedimos al médico que nos de los datos del paciente
resultados = {"Age" : int(input("¿Cuál es tu edad?")),
"Sex" : input("¿Cuál es tu genero?"),
"ChestPainType" : input("¿Tuviste dolor de pecho?"),
"RestingBP" : int(input("¿Cual es tu presion arterial?")),
"Cholesterol" : int(input("¿Cual es tu colesterol?")),
"FastingBS" : int(input("¿Cual es tu nivel de azucar?")),
"RestingECG" : input("¿Cual es tu resultado del electrocardiograma?"),
"MaxHR" : int(input("¿Cual es tu frecuencia cardiaca?")),
"ExerciseAngina" : input("¿Angina inducida?"),
"Oldpeak" : float(input("oldpeak?")),
"ST_Slope" : input("¿Cómo era la pendiente?")}


In [127]:
# convertimos los datos del paciente a dataframe para poder trabajar con ellos. 
df_resultados = pd.DataFrame(resultados, index=[0])
df_resultados

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,23,M,ASY,89,45,0,Normal,98,N,1.0,Flat


In [128]:
# estandarizamos todas las columnas que son de tipo numérico usando el transformer que nos guardamos cuando hicimos el modelo

estandarizadas = pd.DataFrame(estandarizacion.transform(df_resultados[["Age", "RestingBP", "Cholesterol", 
                           "MaxHR", "Oldpeak"]]), columns = ["Age", "RestingBP", "Cholesterol", 
                           "MaxHR", "Oldpeak"])
estandarizadas

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak
0,-3.236379,-2.699359,-4.155098,-1.54655,0.178162


In [129]:
df_resultados.head(1)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,23,M,ASY,89,45,0,Normal,98,N,1.0,Flat


In [130]:
# hacemos el encoding del resto de las variables

chest = pd.DataFrame(encoding_chestpain.transform(df_resultados[["ChestPainType"]]), columns= ["ChestPainType"])
angina = pd.DataFrame(encoding_angina.transform(df_resultados[["ExerciseAngina"]]), columns= ["ExerciseAngina"])
resting = pd.DataFrame(encoding_resting.transform(df_resultados[["RestingECG"]]), columns= ["RestingECG"])
slope = pd.DataFrame(encoding_slope.transform(df_resultados[["ST_Slope"]]), columns= ["ST_Slope"])
fasting = pd.DataFrame(encoding_fasting.transform(df_resultados[["FastingBS"]]), columns= ["FastingBS"])
sex = pd.DataFrame(encoding_sex.transform(df_resultados[["Sex"]]).toarray(), columns=["Sex_F", "Sex_M"])


In [131]:
# unimos todos los conjuntos de datos que hemos ido generando con los transformers
df_final = pd.concat([estandarizadas, sex, chest, angina, resting, slope, fasting], axis = 1)
df_final

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType,ExerciseAngina,RestingECG,ST_Slope,FastingBS
0,-3.236379,-2.699359,-4.155098,-1.54655,0.178162,0.0,1.0,0,0,0,0,0


In [132]:
# cambiamos el orden de las columnas ya que estas deben estar en el mismo orden de como entrenamos al modelo
df_final = df_final.reindex(['Age', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'Sex_F', 'Sex_M'], axis=1)

In [136]:
# hacemos la predicción para nuestro nuevo paciente
modelo.predict(df_final)

array([1])

In [137]:
# ¿Cuál es la probabilidad de que le un infarto al paciente?

modelo.predict_proba(df_final)

array([[0.14088556, 0.85911444]])